# Tarea 2 Modelo de Regresion
## Escoge de entre los diferentes modelos de regresion (lineal, polinomial, knn, decision tree) + regularizacion o normalizacion donde aplique, para crear un modelo con un conjunto de datos de tu eleccion  y utiliza cross validation con un criterio a tu eleccion (R2, RMSE, MSE)  para escoger el mejor modelo para tu criterio.
### El dataset Escogido para la realización de esta practica es el dataset de enfermedades del corazón de el repositorio de UCI. 

In [2]:
#Traemos las librerias necesarias:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
#Traemos la fuente de los datos
from ucimlrepo import fetch_ucirepo 
  
#Obtenemos el conjunto de datos.
heart_disease = fetch_ucirepo(id=45) 
  
# Obtenemos los datos del repositorio.
X = heart_disease.data.features 
y = heart_disease.data.targets 
  
  
# Información de cada una de las variables
print(heart_disease.variables) 



        name     role         type demographic  \
0        age  Feature      Integer         Age   
1        sex  Feature  Categorical         Sex   
2         cp  Feature  Categorical        None   
3   trestbps  Feature      Integer        None   
4       chol  Feature      Integer        None   
5        fbs  Feature  Categorical        None   
6    restecg  Feature  Categorical        None   
7    thalach  Feature      Integer        None   
8      exang  Feature  Categorical        None   
9    oldpeak  Feature      Integer        None   
10     slope  Feature  Categorical        None   
11        ca  Feature      Integer        None   
12      thal  Feature  Categorical        None   
13       num   Target      Integer        None   

                                          description  units missing_values  
0                                                None  years             no  
1                                                None   None             no  
2              

In [3]:
# Mostramos las primeras columnas
print(X.head())
print(y.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  
0  0.0   6.0  
1  3.0   3.0  
2  2.0   7.0  
3  0.0   3.0  
4  0.0   3.0  
   num
0    0
1    2
2    1
3    0
4    0


### La variable con la que buscaremos predecir es "thalach" que hace referencia al mayor pulso cardiaco

In [4]:
target_variable = 'thalach'  
# Extraemos la variable objetivo y la eliminamos de las características
y = X[target_variable]
X = X.drop(columns=[target_variable])

In [5]:
# Mostrar cantidad de datos faltantes antes de eliminarlos
total_missing = X.isnull().sum().sum()
print(f"Total de datos faltantes en el conjunto de características: {total_missing}")
    

Total de datos faltantes en el conjunto de características: 6


In [6]:
 # Eliminamos filas con valores faltantes
data = pd.concat([X, y], axis=1)  # Unimos X e y para eliminar filas faltantes
data = data.dropna()  # Eliminamos filas con valores faltantes

# Separamos X e y nuevamente
X = data.drop(columns=[target_variable])
y = data[target_variable]

In [7]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train)
print(X_test)

     age  sex  cp  trestbps  chol  fbs  restecg  exang  oldpeak  slope   ca  \
277   39    0   3       138   220    0        0      0      0.0      2  0.0   
262   60    0   1       150   240    0        0      0      0.9      1  0.0   
30    69    0   1       140   239    0        0      0      1.8      1  2.0   
22    58    1   2       120   284    0        2      0      1.8      2  0.0   
281   47    1   3       130   253    0        0      0      0.0      1  0.0   
..   ...  ...  ..       ...   ...  ...      ...    ...      ...    ...  ...   
190   50    1   3       129   196    0        0      0      0.0      1  0.0   
71    67    1   4       125   254    1        0      0      0.2      2  2.0   
107   57    1   3       128   229    0        2      0      0.4      2  1.0   
274   59    1   1       134   204    0        0      0      0.8      1  2.0   
103   71    0   3       110   265    1        2      0      0.0      1  1.0   

     thal  
277   3.0  
262   3.0  
30    3.0  
22 

In [8]:
# Inicialización de los modelos
models = {
        "Linear Regression": LinearRegression(),
        "Polynomial Regression": PolynomialFeatures(degree=2),
        "KNN": KNeighborsRegressor(n_neighbors=5),
        "Decision Tree": DecisionTreeRegressor(random_state=42),
        "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42)
    }

In [9]:
# Función para evaluar los modelos
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)  # Entrenamos el modelo
    y_pred = model.predict(X_test)  # Realizamos predicciones sobre el conjunto de prueba
    mse = mean_squared_error(y_test, y_pred)  # Calculamos el error cuadrático medio
    r2 = r2_score(y_test, y_pred)  # Calculamos el coeficiente de determinación R2
    return y_pred, mse, r2

In [10]:
# Evaluación y comparación de los modelos usando validación cruzada
results = {}
predictions = {}
for name, model in models.items():
    if name == "Polynomial Regression":
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train_scaled)
        X_test_poly = poly.transform(X_test_scaled)
        lin_reg = LinearRegression()
        lin_reg.fit(X_train_poly, y_train)
        y_pred, mse, r2 = evaluate_model(lin_reg, X_train_poly, y_train, X_test_poly, y_test)
    else:
        y_pred, mse, r2 = evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test)
    results[name] = {"MSE": mse, "R2": r2}
    predictions[name] = y_pred

In [11]:
# Mostrar los resultados
results_df = pd.DataFrame(results).T
print(results_df)
    
    

                              MSE        R2
Linear Regression      298.085729  0.336530
Polynomial Regression  458.654717 -0.020859
KNN                    288.778667  0.357246
Decision Tree          530.000000 -0.179657
Random Forest          285.921640  0.363605


In [12]:
# Seleccionar el mejor modelo basado en el R2
best_model_name = results_df["R2"].idxmax()
best_model = models[best_model_name]
best_predictions = predictions[best_model_name]
print(f"El mejor modelo es: {best_model_name} con R2: {results_df.loc[best_model_name, 'R2']}")

El mejor modelo es: Random Forest con R2: 0.363604666446975
